In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import re
import time
import csv
import pandas as pd
import numpy as np

In [2]:
df_urls = pd.read_excel('Audi_url.xlsx')
df_urls

,Branch,Adress,Url
0,Audi Zentrum München Hochstraße,"Hochstraße 15, 81669 München",https://www.google.com/maps/place/Audi+Zentrum...
1,Audi Zentrum München Albrechtstraße,"Albrechtstraße 16, 80636 München",https://www.google.com/maps/place/Audi+Zentrum...
2,Audi Trudering München,"Wasserburger Landstraße 69, 81825 München",https://www.google.com/maps/place/Audi+Truderi...
3,Audi München Starnberg,"Münchner Str. 25, 82319 Starnberg",https://www.google.com/maps/place/Audi+M%C3%BC...
4,Audi plus Zentrum München,"Heisenbergstraße 4, 85386 Eching",https://www.google.com/maps/place/Audi+plus+Ze...
5,Audi Zentrum Ingolstadt,"Neuburger Str. 75, 85057 Ingolstadt",https://www.google.com/maps/place/Audi+Zentrum...
6,Audi Autohaus Neumayr GmbH & Co. KG,"Olchinger Str. 150, 82194 Gröbenzell",https://www.google.com/maps/place/Audi+Autohau...
7,Audi Autohaus E. Hoppe GmbH,"Allacher Str. 250 - 252, 80999 München",https://www.google.com/maps/place/Autohaus+E.+...
8,Audi Autohaus Christl GmbH & Co. KG,"Filchnerstraße 86, 81476 München",https://www.google.com/maps/place/Autohaus+Chr...
9,Audi Autohaus Ebersberg / AUDI,"Gewerbepark Nord-Ost 2, 85560 Ebersberg",https://www.google.com/maps/place/Autohaus+Ebe...


In [ ]:
results = pd.DataFrame()

for i, row in df_urls.iterrows():
    branch = row['Branch']
    url = row['Url']

    # Start a new session with the browser via a driver.
    op = webdriver.ChromeOptions()
    op.add_argument("--start-maximized")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=op)
    wait = WebDriverWait(driver, 10)
    driver.get(url)
    time.sleep(10)

    # Click on "Cookies" Button.
    try:
        driver.find_element_by_xpath('//button[@aria-label=\'In die Verwendung von Cookies und anderen Daten zu den beschriebenen Zwecken einwilligen\']').click()
        time.sleep(10)
    except:
        pass
    
    num_reviews = 0
    i = 0
    try:
        while True:
            scrollable_div = driver.find_element_by_xpath(
             '//div[@class=\'m6QErb DxyBCb kA9KIf dS8AEf\']'
                        )
            driver.execute_script(
                           'arguments[0].scrollTop = arguments[0].scrollHeight', 
                            scrollable_div
                        )
            time.sleep(2)

            response = BeautifulSoup(driver.page_source, 'html.parser')
            rlist = response.find_all('div', class_='jftiEf L6Bbsd fontBodyMedium')

            if i % 10 == 0:
                if len(rlist) == num_reviews:
                    break
                else:
                    num_reviews = len(rlist)

            i += 1
    except:
        pass
        
    # Extend reviews
    extends = driver.find_elements_by_xpath('//button[@aria-label=\' See more \']')
    for e in extends:
        e.click()
    time.sleep(30)
    
    # Get all reviews
    response = BeautifulSoup(driver.page_source, 'html.parser')
    rlist = response.find_all('div', class_='jftiEf L6Bbsd fontBodyMedium')
    rdict = {'Username':[], 'Description':[], 'Number of reviews':[], 
             'Profile URL':[], 'Image URL':[], 
             'Review':[], 'Branch':[], 'Rating':[], 'Date':[]}

    for r in rlist:
        username = r.find('div', 
                          class_='d4r55').find('span').text
        try:
            if r.find('div', class_='RfnDt').find_all('span')[0]['style'] == 'display:none':
                description = ''
        except:
            try:
                description = r.find('div', class_='RfnDt').find_all('span')[0].text
            except:
                pass

        num_r = 0
        try:
            num_r = int(r.find('div', class_='RfnDt').find_all('span')[1].text.split(' ')[0])
        except:
            pass
        profile_url = r.find('div', 
                          class_='WNxzHc qLhwHc').find('a')['href']
        img_url = r.find('a', 
                          class_='WEBjve').find('img')['src']
        
        try:
            review_text = r.find('span', class_='wiI7pd').text
        except Exception:
            review_text = None
        rating = r.find('span', class_='kvMYJc')['aria-label']
        rel_date = r.find('span', class_='rsqaWe').text

        rdict['Username'].append(username)
        rdict['Description'].append(description)
        rdict['Number of reviews'].append(num_r)
        rdict['Profile URL'].append(profile_url)
        rdict['Image URL'].append(img_url)
        rdict['Review'].append(review_text)
        rdict['Rating'].append(rating)
        rdict['Branch'].append(branch)
        rdict['Date'].append(rel_date)
        
    driver.quit()
        
    results = results.append(pd.DataFrame(rdict, columns = rdict.keys()), ignore_index = True)
    
    print(f"{branch} - {len(rlist)} reviews")

results['Rating'] = results['Rating'].apply(lambda x: int(x[1]))

In [ ]:
results

In [ ]:
results.to_excel('auidi_rew.xlsx', index = False)